Import and initializing market data

In [1]:
from instruments.interestRateInstrument.interestRateSwap import InterestRateSwap
from utilities.Enums import SwapDirection, Currency
from marketdata import init_marketdata


In [2]:
notional = 1000000
fixed_rate = 0.025
float_spread = 0.004
currency = Currency.USD
ttstart = 5/365
ttend = 10
swap = InterestRateSwap(swapDirection=SwapDirection.PAYER, notional=notional, currency=currency, timeToSwapStart_in_days=ttstart, timeToSwapEnd_in_days=ttend, fixed_rate=fixed_rate, float_spread=float_spread )


First we need to get the original price of the swap since we want to calculate a finite difference

In [3]:
pv0 = swap.get_price()
pv0

-38.87376837231966

Let's try to manipulate the market data used to price the swap. Currently all swaps are using a common ois curve handle for pricing.

In [4]:
from marketdata.interestRateCurves import flat_ois_quote

bp = 1/10000 
flat_ois_quote.setValue(flat_ois_quote.value()+bp)

reprice with shifted data

In [5]:
pv1 = swap.get_price()
pv1

-38.873617196847704

calculate finite difference to get sensitivity

In [6]:
sensitivity = (pv1-pv0)/bp
sensitivity

1.5117547195586667

Calculate DV01 because that is actually used more frequently in the interest world

In [7]:
DV01 = pv1-pv0
DV01

0.00015117547195586667

Reset ois curve to its original value and check if the trade has its original price again

In [8]:
flat_ois_quote.setValue(flat_ois_quote.value()-bp)
pv2 = swap.get_price()
pv2 == pv0


True